In [0]:
# -----------------------------------------------------------
# 4. Entrenamiento simple del modelo
# -----------------------------------------------------------
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, classification_report

In [0]:
from mlflow.models import infer_signature

In [0]:

# =========================================================
# 0) Base mensual consolidada
# =========================================================
features_final = spark.table("esan.ventas.dataset_entrenamiento")

In [0]:
# Convertimos a Pandas para usar con scikit-learn
features_pd = features_final.toPandas()

# Definimos las features (columnas) y el target para el modelo
feature_list = [
    'frecuencia_1m', 'cantidad_1m', 'tasa', 'ultima_compra_1m', 'monto_total', 
    'avg_monto', 'max_monto'
]
X = features_pd[feature_list]
y = features_pd["flg_churn"]

In [0]:
# Dividimos la data en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [0]:
with mlflow.start_run():
    # Creamos y entrenamos un modelo simple (Random Forest)
    model_train = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, class_weight="balanced")
    model_train.fit(X_train, y_train)
    
    # Realizamos predicciones en el conjunto de prueba
    y_pred_train = model_train.predict(X_test)
    
    signature = infer_signature(X_test, y_test)  # Inferimos la firma con datos de prueba y predicciones

    # Calculamos métricas de desempeño
    f1 = f1_score(y_test, y_pred_train)
    accuracy = accuracy_score(y_test, y_pred_train)
    print(f"F1 Score (entrenamiento): {f1:.4f}")
    print(f"Accuracy (entrenamiento): {accuracy:.4f}")
    print("Reporte de clasificación (entrenamiento):\n", classification_report(y_test, y_pred_train))
    
    # Registramos parámetros y el modelo en MLflow
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 10)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("accuracy", accuracy)
    #mlflow.sklearn.log_model(model_train, "modelo_simple")

    mlflow.sklearn.log_model(
        sk_model=model_train,
        artifact_path="modelo_simple",
        signature=signature,  # Agregamos la firma aquí
        input_example=X_test.iloc[:5]  # Opcional pero recomendable para referencia
    )
    # Opcional: registrar el modelo en el Model Registry
mlflow.end_run()